In [10]:
#export

import base64
from typing import Optional

from algosdk.future.transaction import SignedTransaction
from algosdk.v2client import algod


class NetworkInteraction:

    @staticmethod
    def wait_for_confirmation(client: algod.AlgodClient, txid, log=True):
        """
        Utility function to wait until the transaction is
        confirmed before proceeding.
        """
        print('1:NetworkInteraction.wait_for_confirmation')
        last_round = client.status().get('last-round')
        txinfo = client.pending_transaction_info(txid)
        while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
            if log:
                print("Waiting for confirmation")
            last_round += 1
            client.status_after_block(last_round)
            txinfo = client.pending_transaction_info(txid)
        if log:
            print(f"Transaction {txid} confirmed in round {txinfo.get('confirmed-round')}.")
        return txinfo

    @staticmethod
    def get_default_suggested_params(client: algod.AlgodClient):
        """
        Gets default suggested params with flat transaction fee and fee amount of 1000.
        :param client:
        :return:
        """
        print('1:NetworkInteraction.get_default_suggested_params')
        suggested_params = client.suggested_params()

        suggested_params.flat_fee = True
        suggested_params.fee = 1000

        return suggested_params

    @staticmethod
    def submit_asa_creation(client: algod.AlgodClient, transaction: SignedTransaction) -> Optional[int]:
        """
        Submits a ASA creation transaction to the network. If the transaction is successful the ASA's id is returned.
        :param client:
        :param transaction:
        :return:
        """
        print('1:NetworkInteraction.submit_asa_creation')
        txid = client.send_transaction(transaction)

        NetworkInteraction.wait_for_confirmation(client, txid)

        try:
            ptx = client.pending_transaction_info(txid)
            return ptx["asset-index"]
        except Exception as e:
            # TODO: Proper logging needed.
            print(e)
            print('Unsuccessful creation of Algorand Standard Asset.')

    @staticmethod
    def submit_transaction(client: algod.AlgodClient, transaction: SignedTransaction, log=True) -> Optional[str]:
        print('1:NetworkInteraction.submit_transaction')
        txid = client.send_transaction(transaction)

        NetworkInteraction.wait_for_confirmation(client, txid, log)

        return txid

    @staticmethod
    def compile_program(client: algod.AlgodClient, source_code):
        """
        :param client: algorand client
        :param source_code: teal source code
        :return:
            Decoded byte program
        """
        print('1:NetworkInteraction.compile_program')
        compile_response = client.compile(source_code)
        return base64.b64decode(compile_response['result'])

In [11]:
#export

import base64
from algosdk.v2client import algod
from algosdk.future import transaction as algo_txn
from typing import List, Any, Optional, Union
from algosdk import account as algo_acc
from algosdk.future.transaction import Transaction, SignedTransaction


def get_default_suggested_params(client: algod.AlgodClient):
    """
    Gets default suggested params with flat transaction fee and fee amount of 1000.
    :param client:
    :return:
    """
    print('2:get_default_suggested_params')
    suggested_params = client.suggested_params()

    suggested_params.flat_fee = True
    suggested_params.fee = 1000

    return suggested_params


class ApplicationTransactionRepository:
    """
    Initializes transaction related to applications.
    """

    @classmethod
    def create_application(cls,
                           client: algod.AlgodClient,
                           creator_private_key: str,
                           approval_program: bytes,
                           clear_program: bytes,
                           global_schema: algo_txn.StateSchema,
                           local_schema: algo_txn.StateSchema,
                           app_args: Optional[List[Any]],
                           sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Initiates a transaction that represents application creation. The transaction is optionally signed using the
        provided private key.
        :param client: algorand client.
        :param creator_private_key: private key of the creator of the application.
        :param approval_program: encoded source code in bytes for the approval program.
        :param clear_program: encoded source code in bytes for the clear program.
        :param global_schema: global schema for the application.
        :param local_schema: local schema for the application.
        :param app_args: list of arguments for the application.
        :param sign_transaction: boolean value that determines whether the created transaction should be signed or not.
        :return:
            Returns SignedTransaction or Transaction depending on the boolean property sign_transaction.
        """
        print('2:ApplicationTransactionRepository.create_application')
        creator_address = algo_acc.address_from_private_key(private_key=creator_private_key)
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.ApplicationCreateTxn(sender=creator_address,
                                            sp=suggested_params,
                                            on_complete=algo_txn.OnComplete.NoOpOC.real,
                                            approval_program=approval_program,
                                            clear_program=clear_program,
                                            global_schema=global_schema,
                                            local_schema=local_schema,
                                            app_args=app_args)

        if sign_transaction:
            txn = txn.sign(private_key=creator_private_key)

        return txn

    @classmethod
    def call_application(cls,
                         client: algod.AlgodClient,
                         caller_private_key: str,
                         app_id: int,
                         on_complete: algo_txn.OnComplete,
                         app_args: Optional[List[Any]] = None,
                         sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Creates a transaction that represents an application call.
        :param client: algorand client.
        :param caller_private_key: the private key of the caller of the application.
        :param app_id: the application id which identifies the app.
        :param on_complete: Type of the application call.
        :param app_args: Arguments of the application.
        :param sign_transaction: boolean value that determines whether the created transaction should be signed or not.
        :return:
        Returns SignedTransaction or Transaction depending on the boolean property sign_transaction.
        """
        print('2:ApplicationTransactionRepository.call_application')
        caller_address = algo_acc.address_from_private_key(private_key=caller_private_key)
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.ApplicationCallTxn(sender=caller_address,
                                          sp=suggested_params,
                                          index=app_id,
                                          app_args=app_args,
                                          on_complete=on_complete)

        if sign_transaction:
            txn = txn.sign(private_key=caller_private_key)

        return txn


class ASATransactionRepository:
    """
    Initializes transactions related to Algorand Standard Assets
    """

    @classmethod
    def create_asa(cls,
                   client: algod.AlgodClient,
                   creator_private_key: str,
                   unit_name: str,
                   asset_name: str,
                   total: int,
                   decimals: int,
                   note: Optional[bytes] = None,
                   manager_address: Optional[str] = None,
                   reserve_address: Optional[str] = None,
                   freeze_address: Optional[str] = None,
                   clawback_address: Optional[str] = None,
                   url: Optional[str] = None,
                   default_frozen: bool = False,
                   sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param creator_private_key:
        :param unit_name:
        :param asset_name:
        :param total:
        :param decimals:
        :param note:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param url:
        :param default_frozen:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.create_application')

        suggested_params = get_default_suggested_params(client=client)

        creator_address = algo_acc.address_from_private_key(private_key=creator_private_key)

        txn = algo_txn.AssetConfigTxn(sender=creator_address,
                                      sp=suggested_params,
                                      total=total,
                                      default_frozen=default_frozen,
                                      unit_name=unit_name,
                                      asset_name=asset_name,
                                      manager=manager_address,
                                      reserve=reserve_address,
                                      freeze=freeze_address,
                                      clawback=clawback_address,
                                      url=url,
                                      decimals=decimals,
                                      note=note)

        if sign_transaction:
            txn = txn.sign(private_key=creator_private_key)

        return txn

    @classmethod
    def create_non_fungible_asa(cls,
                                client: algod.AlgodClient,
                                creator_private_key: str,
                                unit_name: str,
                                asset_name: str,
                                note: Optional[bytes] = None,
                                manager_address: Optional[str] = None,
                                reserve_address: Optional[str] = None,
                                freeze_address: Optional[str] = None,
                                clawback_address: Optional[str] = None,
                                url: Optional[str] = None,
                                default_frozen: bool = False,
                                sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param creator_private_key:
        :param unit_name:
        :param asset_name:
        :param note:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param url:
        :param default_frozen:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.create_non_fungible_asa')

        return ASATransactionRepository.create_asa(client=client,
                                                   creator_private_key=creator_private_key,
                                                   unit_name=unit_name,
                                                   asset_name=asset_name,
                                                   total=1,
                                                   decimals=0,
                                                   note=note,
                                                   manager_address=manager_address,
                                                   reserve_address=reserve_address,
                                                   freeze_address=freeze_address,
                                                   clawback_address=clawback_address,
                                                   url=url,
                                                   default_frozen=default_frozen,
                                                   sign_transaction=sign_transaction)

    @classmethod
    def asa_opt_in(cls,
                   client: algod.AlgodClient,
                   sender_private_key: str,
                   asa_id: int,
                   sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Opts-in the sender's account to the specified asa with an id: asa_id.
        :param client:
        :param sender_private_key:
        :param asa_id:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.asa_opt_in')

        suggested_params = get_default_suggested_params(client=client)
        sender_address = algo_acc.address_from_private_key(sender_private_key)

        txn = algo_txn.AssetTransferTxn(sender=sender_address,
                                        sp=suggested_params,
                                        receiver=sender_address,
                                        amt=0,
                                        index=asa_id)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

    @classmethod
    def asa_transfer(cls,
                     client: algod.AlgodClient,
                     sender_address: str,
                     receiver_address: str,
                     asa_id: int,
                     amount: int,
                     revocation_target: Optional[str],
                     sender_private_key: Optional[str],
                     sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        :param client:
        :param sender_address:
        :param receiver_address:
        :param asa_id:
        :param amount:
        :param revocation_target:
        :param sender_private_key:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.asa_transfer')
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.AssetTransferTxn(sender=sender_address,
                                        sp=suggested_params,
                                        receiver=receiver_address,
                                        amt=amount,
                                        index=asa_id,
                                        revocation_target=revocation_target)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

    @classmethod
    def change_asa_management(cls,
                              client: algod.AlgodClient,
                              current_manager_pk: str,
                              asa_id: int,
                              manager_address: Optional[str] = None,
                              reserve_address: Optional[str] = None,
                              freeze_address: Optional[str] = None,
                              clawback_address: Optional[str] = None,
                              strict_empty_address_check: bool = True,
                              sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Changes the management properties of a given ASA.
        :param client:
        :param current_manager_pk:
        :param asa_id:
        :param manager_address:
        :param reserve_address:
        :param freeze_address:
        :param clawback_address:
        :param strict_empty_address_check:
        :param sign_transaction:
        :return:
        """
        print('2:ASATransactionRepository.change_asa_management')

        params = get_default_suggested_params(client=client)

        current_manager_address = algo_acc.address_from_private_key(private_key=current_manager_pk)

        txn = algo_txn.AssetConfigTxn(
            sender=current_manager_address,
            sp=params,
            index=asa_id,
            manager=manager_address,
            reserve=reserve_address,
            freeze=freeze_address,
            clawback=clawback_address,
            strict_empty_address_check=strict_empty_address_check)

        if sign_transaction:
            txn = txn.sign(private_key=current_manager_pk)

        return txn


class PaymentTransactionRepository:

    @classmethod
    def payment(cls,
                client: algod.AlgodClient,
                sender_address: str,
                receiver_address: str,
                amount: int,
                sender_private_key: Optional[str],
                sign_transaction: bool = True) -> Union[Transaction, SignedTransaction]:
        """
        Creates a payment transaction in ALGOs.
        :param client:
        :param sender_address:
        :param receiver_address:
        :param amount:
        :param sender_private_key:
        :param sign_transaction:
        :return:
        """
        print('2:PaymentTransactionRepository.payment')
        suggested_params = get_default_suggested_params(client=client)

        txn = algo_txn.PaymentTxn(sender=sender_address,
                                  sp=suggested_params,
                                  receiver=receiver_address,
                                  amt=amount)

        if sign_transaction:
            txn = txn.sign(private_key=sender_private_key)

        return txn

In [12]:
#export

from pyteal import *


def game_funds_escrow(app_id: int):
    print('3:game_funds_escrow')

    win_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address())
    ])

    tie_refund = Seq([
        Assert(Gtxn[0].application_id() == Int(app_id)),
        Assert(Gtxn[1].fee() <= Int(1000)),
        Assert(Gtxn[1].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[1].rekey_to() == Global.zero_address()),
        Assert(Gtxn[2].fee() <= Int(1000)),
        Assert(Gtxn[2].asset_close_to() == Global.zero_address()),
        Assert(Gtxn[2].rekey_to() == Global.zero_address())
    ])

    return Seq([
        Cond(
            [Global.group_size() == Int(2), win_refund],
            [Global.group_size() == Int(3), tie_refund],
        ),
        Return(Int(1))
    ])

In [29]:
import main

approval_program = main.arrow_6

def clear_program():
    print('5:clear_program')
    return Return(Int(1))

def number_of_int():
    print('5:AppVariables.number_of_int')
    return 5

def number_of_str():
    print('5:AppVariables.number_of_str')
    return 4

In [30]:
#export

from algosdk import logic as algo_logic
from algosdk.future import transaction as algo_txn
from pyteal import compileTeal, Mode


class GameEngineService:
    """
    Engine that defines the interaction and initialization of the Tic-Tac-Toe DApp.
    """

    def __init__(self,
                 app_creator_pk,
                 app_creator_address,
                 player_x_pk,
                 player_x_address,
                 player_o_pk,
                 player_o_address):
        print('4:GameEngineService.__init__')

        self.app_creator_pk = app_creator_pk
        self.app_creator_address = app_creator_address
        self.player_x_pk = player_x_pk
        self.player_x_address = player_x_address
        self.player_o_pk = player_o_pk
        self.player_o_address = player_o_address
        self.teal_version = 4

        self.approval_program_code = approval_program()
        self.clear_program_code = clear_program()

        self.app_id = None
        self.escrow_fund_address = None
        self.escrow_fund_program_bytes = None

    def deploy_application(self, client):
        """
        Creates and sends the transaction to the network that does the initialization of the Tic-Tac-Toe game.
        :param client:
        :return:
        """
        print('4:GameEngineService.deploy_application')
        approval_program_compiled = compileTeal(approval_program(),
                                                mode=Mode.Application,
                                                version=self.teal_version)

        clear_program_compiled = compileTeal(clear_program(),
                                             mode=Mode.Application,
                                             version=self.teal_version)

        approval_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                    source_code=approval_program_compiled)

        clear_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                 source_code=clear_program_compiled)

        global_schema = algo_txn.StateSchema(num_uints=number_of_int(),
                                             num_byte_slices=number_of_str())

        local_schema = algo_txn.StateSchema(num_uints=0,
                                            num_byte_slices=0)

        app_transaction = ApplicationTransactionRepository.create_application(client=client,
                                                                              creator_private_key=self.app_creator_pk,
                                                                              approval_program=approval_program_bytes,
                                                                              clear_program=clear_program_bytes,
                                                                              global_schema=global_schema,
                                                                              local_schema=local_schema,
                                                                              app_args=None)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=app_transaction,
                                                      log=False)

        transaction_response = client.pending_transaction_info(tx_id)

        self.app_id = transaction_response['application-index']
        print(f"Tic-Tac-Toe application deployed with the application_id: {self.app_id}")

        return f"Tic-Tac-Toe application deployed with the application_id: {self.app_id}"

    def start_game(self, client):
        """
        Atomic transfer of 3 transactions:
        - 1. Application call
        - 2. Payment from the Player X address to the Escrow fund address
        - 3. Payment from the Player O address to the Escrow fund address
        :param client:
        :return:
        """
        print('4:GameEngineService.start_game')
        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        if self.escrow_fund_address is not None or self.escrow_fund_program_bytes is not None:
            raise ValueError('The game has already started!')

        escrow_fund_program_compiled = compileTeal(game_funds_escrow(app_id=self.app_id),
                                                   mode=Mode.Signature,
                                                   version=self.teal_version)

        self.escrow_fund_program_bytes = NetworkInteraction.compile_program(client=client,
                                                                            source_code=escrow_fund_program_compiled)

        self.escrow_fund_address = algo_logic.address(self.escrow_fund_program_bytes)

        player_x_funding_txn = PaymentTransactionRepository.payment(client=client,
                                                                    sender_address=self.player_x_address,
                                                                    receiver_address=self.escrow_fund_address,
                                                                    amount=1000000,
                                                                    sender_private_key=None,
                                                                    sign_transaction=False)

        player_o_funding_txn = PaymentTransactionRepository.payment(client=client,
                                                                    sender_address=self.player_o_address,
                                                                    receiver_address=self.escrow_fund_address,
                                                                    amount=1000000,
                                                                    sender_private_key=None,
                                                                    sign_transaction=False)

        app_args = [
            "SetupPlayers"
        ]

        app_initialization_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=self.app_creator_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_initialization_txn,
                                           player_x_funding_txn,
                                           player_o_funding_txn])

        app_initialization_txn.group = gid
        player_x_funding_txn.group = gid
        player_o_funding_txn.group = gid

        app_initialization_txn_signed = app_initialization_txn.sign(self.app_creator_pk)
        player_x_funding_txn_signed = player_x_funding_txn.sign(self.player_x_pk)
        player_o_funding_txn_signed = player_o_funding_txn.sign(self.player_o_pk)

        signed_group = [app_initialization_txn_signed,
                        player_x_funding_txn_signed,
                        player_o_funding_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"Game started with the transaction_id: {txid}")

        return f"Game started with the transaction_id: {txid}"

    def play_action(self, client, player_id: str, action_position: int):
        """
        Application call transaction that performs an action for the specified player at the specified action position.
        :param client:
        :param player_id: "X" or "O"
        :param action_position: action position in the range of [0, 8]
        :return:
        """
        print('4:GameEngineService.play_action')
        if player_id != "X" and player_id != "O":
            raise ValueError('Invalid player id! The player_id should be X or O.')

        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        app_args = [
            "ActionMove",
            action_position]

        player_pk = self.player_x_pk if player_id == "X" else self.player_o_pk

        app_initialization_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=player_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=app_initialization_txn,
                                                      log=False)

        print(f"{player_id} has been put at position {action_position} in transaction with id: {tx_id}")

        return f"{player_id} has been put at position {action_position} in transaction with id: {tx_id}"

    def fund_escrow(self, client):
        """
        Funding the escrow address in order to handle the transactions fees for refunding.
        :param client:
        :return:
        """
        print('4:GameEngineService.fund_escrow')
        fund_escrow_txn = PaymentTransactionRepository.payment(client=client,
                                                               sender_address=self.app_creator_address,
                                                               receiver_address=self.escrow_fund_address,
                                                               amount=1000000,
                                                               sender_private_key=self.app_creator_pk,
                                                               sign_transaction=True)

        tx_id = NetworkInteraction.submit_transaction(client,
                                                      transaction=fund_escrow_txn,
                                                      log=False)

        print(f'Escrow address has been funded in transaction with id: {tx_id}')
        return f'Escrow address has been funded in transaction with id: {tx_id}'

    def win_money_refund(self, client, player_id: str):
        """
        Atomic transfer of 2 transactions:
        1. Application call
        2. Payment from the Escrow account to winner address either PlayerX or PlayerO.
        :param client:
        :param player_id: "X" or "O".
        :return:
        """
        print('4:GameEngineService.win_money_refund')
        if player_id != "X" and player_id != "O":
            raise ValueError('Invalid player id! The player_id should be X or O.')

        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        player_pk = self.player_x_pk if player_id == "X" else self.player_o_pk
        player_address = self.player_x_address if player_id == "X" else self.player_o_address

        app_args = [
            "MoneyRefund"
        ]

        app_withdraw_call_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=player_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        refund_txn = PaymentTransactionRepository.payment(client=client,
                                                          sender_address=self.escrow_fund_address,
                                                          receiver_address=player_address,
                                                          amount=2000000,
                                                          sender_private_key=None,
                                                          sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_withdraw_call_txn,
                                           refund_txn])

        app_withdraw_call_txn.group = gid
        refund_txn.group = gid

        app_withdraw_call_txn_signed = app_withdraw_call_txn.sign(player_pk)

        refund_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_txn_signed = algo_txn.LogicSigTransaction(refund_txn, refund_txn_logic_signature)

        signed_group = [app_withdraw_call_txn_signed,
                        refund_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"The winning money have been refunded to the player {player_id} in the transaction with id: {txid}")
        return f"The winning money have been refunded to the player {player_id} in the transaction with id: {txid}"

    def tie_money_refund(self, client):
        """
        Atomic transfer of 3 transactions:
        1. Application call
        2. Payment from the escrow address to the PlayerX address.
        3. Payment from the escrow address to the PlayerO address.
        :param client:
        :return:
        """
        print('4:GameEngineService.tie_money_refund')
        if self.app_id is None:
            raise ValueError('The application has not been deployed')

        app_args = [
            "MoneyRefund"
        ]

        app_withdraw_call_txn = \
            ApplicationTransactionRepository.call_application(client=client,
                                                              caller_private_key=self.app_creator_pk,
                                                              app_id=self.app_id,
                                                              on_complete=algo_txn.OnComplete.NoOpOC,
                                                              app_args=app_args,
                                                              sign_transaction=False)

        refund_player_x_txn = PaymentTransactionRepository.payment(client=client,
                                                                   sender_address=self.escrow_fund_address,
                                                                   receiver_address=self.player_x_address,
                                                                   amount=1000000,
                                                                   sender_private_key=None,
                                                                   sign_transaction=False)

        refund_player_o_txn = PaymentTransactionRepository.payment(client=client,
                                                                   sender_address=self.escrow_fund_address,
                                                                   receiver_address=self.player_o_address,
                                                                   amount=1000000,
                                                                   sender_private_key=None,
                                                                   sign_transaction=False)

        gid = algo_txn.calculate_group_id([app_withdraw_call_txn,
                                           refund_player_x_txn,
                                           refund_player_o_txn])

        app_withdraw_call_txn.group = gid
        refund_player_x_txn.group = gid
        refund_player_o_txn.group = gid

        app_withdraw_call_txn_signed = app_withdraw_call_txn.sign(self.app_creator_pk)

        refund_player_x_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_player_x_txn_signed = \
            algo_txn.LogicSigTransaction(refund_player_x_txn, refund_player_x_txn_logic_signature)

        refund_player_o_txn_logic_signature = algo_txn.LogicSig(self.escrow_fund_program_bytes)
        refund_player_o_txn_signed = \
            algo_txn.LogicSigTransaction(refund_player_o_txn, refund_player_o_txn_logic_signature)

        signed_group = [app_withdraw_call_txn_signed,
                        refund_player_x_txn_signed,
                        refund_player_o_txn_signed]

        txid = client.send_transactions(signed_group)

        print(f"The initial bet money have been refunded to the players in the transaction with id: {txid}")

        return f"The initial bet money have been refunded to the players in the transaction with id: {txid}"

# testnet

In [16]:
accounts = [
 {'private_key': 'zhzM1byIvHXktgsDT8RodEh9o6LThrAG/Q1DZguXczasayFqplinX2fz1MkmljvkSdBo996hACdKf1jaJxJAVg==',
  'address': 'VRVSC2VGLCTV6Z7T2TESNFR34RE5A2HX32QQAJ2KP5MNUJYSIBLFOY3ISU',
  'mnemonic': 'oil corn helmet mesh jungle buddy rescue arm there bachelor crucial mansion whisper potato other man stock village manage goddess bitter total receive ability second'},
 {'private_key': 'Jv/fyfKxKr40mdocZE4hHBE84V8iLPBC8wjkEF3SJTX+JuCVO5tLjaQO6I3Dqr9Jqb2GREafYHsvPrDvNNdWOA==',
  'address': '7YTOBFJ3TNFY3JAO5CG4HKV7JGU33BSEI2PWA6ZPH2YO6NGXKY4EGGMXLU',
  'mnemonic': 'tooth lend gossip busy fever convince situate surround logic define awful balcony join weapon base club fun snack afford atom riot innocent powder ability rib'},
 {'private_key': 'FtV6JAmeHHZHsp/UR2Iakg4mDQTt2LXxPHVSKPFX9d6koSieghGVIl+GEjKoKQHbupZ7fgbzJuYC+Gl9pVWA+A==',
  'address': 'USQSRHUCCGKSEX4GCIZKQKIB3O5JM636A3ZSNZQC7BUX3JKVQD4IQMSVKY',
  'mnemonic': 'people twice mutual they decrease talk mutual panic whisper math artwork truck equal addict habit suggest cute video dentist lunch time field waste absorb funny'}]
accounts

[{'private_key': 'zhzM1byIvHXktgsDT8RodEh9o6LThrAG/Q1DZguXczasayFqplinX2fz1MkmljvkSdBo996hACdKf1jaJxJAVg==',
  'address': 'VRVSC2VGLCTV6Z7T2TESNFR34RE5A2HX32QQAJ2KP5MNUJYSIBLFOY3ISU',
  'mnemonic': 'oil corn helmet mesh jungle buddy rescue arm there bachelor crucial mansion whisper potato other man stock village manage goddess bitter total receive ability second'},
 {'private_key': 'Jv/fyfKxKr40mdocZE4hHBE84V8iLPBC8wjkEF3SJTX+JuCVO5tLjaQO6I3Dqr9Jqb2GREafYHsvPrDvNNdWOA==',
  'address': '7YTOBFJ3TNFY3JAO5CG4HKV7JGU33BSEI2PWA6ZPH2YO6NGXKY4EGGMXLU',
  'mnemonic': 'tooth lend gossip busy fever convince situate surround logic define awful balcony join weapon base club fun snack afford atom riot innocent powder ability rib'},
 {'private_key': 'FtV6JAmeHHZHsp/UR2Iakg4mDQTt2LXxPHVSKPFX9d6koSieghGVIl+GEjKoKQHbupZ7fgbzJuYC+Gl9pVWA+A==',
  'address': 'USQSRHUCCGKSEX4GCIZKQKIB3O5JM636A3ZSNZQC7BUX3JKVQD4IQMSVKY',
  'mnemonic': 'people twice mutual they decrease talk mutual panic whisper math artwork

In [31]:
game_engine_fixed = GameEngineService(app_creator_pk=accounts[0]['private_key'],
                                app_creator_address=accounts[0]['address'],
                                player_x_pk=accounts[1]['private_key'],
                                player_x_address=accounts[1]['address'],
                                player_o_pk=accounts[2]['private_key'],
                                player_o_address=accounts[2]['address'])
game_engine_fixed

4:GameEngineService.__init__
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program


In [18]:
# import algosdk

# # client = get_client()
# # indexer = get_indexer()

# acc_pk, acc_address = algosdk.account.generate_account()
# player_x_pk, player_x_address = algosdk.account.generate_account()
# player_o_pk, player_o_address = algosdk.account.generate_account()

# # https://bank.testnet.algorand.network
# [acc_pk, acc_address, player_x_pk, player_x_address, player_o_pk, player_o_address]

In [19]:
# game_engine = GameEngineService(app_creator_pk=acc_pk,
#                                 app_creator_address=acc_address,
#                                 player_x_pk=player_x_pk,
#                                 player_x_address=player_x_address,
#                                 player_o_pk=player_o_pk,
#                                 player_o_address=player_o_address)
# game_engine

In [32]:
token = 'aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa'
address = "http://localhost:4001"

def get_client():
    return algod.AlgodClient(token, address, headers={'X-Api-key': token})
client = get_client()

In [34]:
app_deployment_txn_log = game_engine_fixed.deploy_application(client)
app_deployment_txn_log

4:GameEngineService.deploy_application
application_start
initialize_players_logic
play_action_logic
has_player_won
has_player_won
is_tie
money_refund_logic
app_initialization_logic
5:clear_program
1:NetworkInteraction.compile_program
1:NetworkInteraction.compile_program
5:AppVariables.number_of_int
5:AppVariables.number_of_str
2:ApplicationTransactionRepository.create_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
Tic-Tac-Toe application deployed with the application_id: 90667084


'Tic-Tac-Toe application deployed with the application_id: 90667084'

In [35]:
start_game_txn_log = game_engine_fixed.start_game(client)
start_game_txn_log

4:GameEngineService.start_game
3:game_funds_escorw
1:NetworkInteraction.compile_program
2:PaymentTransactionRepository.payment
2:get_default_suggested_params
2:PaymentTransactionRepository.payment
2:get_default_suggested_params
2:ApplicationTransactionRepository.call_application
2:get_default_suggested_params
Game started with the transaction_id: 4UR2QISIYAAOH6MD7MSKWSD2FT5FDFESOVG324N4EJJDJN3RBVEA


'Game started with the transaction_id: 4UR2QISIYAAOH6MD7MSKWSD2FT5FDFESOVG324N4EJJDJN3RBVEA'

In [36]:
game_actions = [
    ("X", 0),
    ("O", 2),
    ("X", 4),
    ("O", 8),
    ("X", 6),
    ("O", 5),
]

for player_id, action_position in game_actions:
    game_engine_fixed.play_action(client=client,
                            player_id=player_id,
                            action_position=action_position)

4:GameEngineService.play_action
2:ApplicationTransactionRepository.call_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
X has been put at position 0 in transaction with id: TKYBCYF6QSZEJCAXNO56GMAMXWS5QQQKO2GMN3MDELOS4W3KW2VA
4:GameEngineService.play_action
2:ApplicationTransactionRepository.call_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
O has been put at position 2 in transaction with id: OTPNUCJJE3GNCQ2UXFQFRGDK37475W3PUD3NTD45TAPT4BJQ5MBQ
4:GameEngineService.play_action
2:ApplicationTransactionRepository.call_application
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
X has been put at position 4 in transaction with id: UFXJA7GQQTRIXDXSJ5SBJKSEU3QC5RZB7IDMDIDBOSI5KRDBHP5Q
4:GameEngineService.play_action
2:ApplicationTransactionRepository.call_application
2:ge

In [37]:
game_engine_fixed.fund_escrow(client=client)

4:GameEngineService.fund_escrow
2:PaymentTransactionRepository.payment
2:get_default_suggested_params
1:NetworkInteraction.submit_transaction
1:NetworkInteraction.wait_for_confirmation
Escrow address has been funded in transaction with id: ES74ENJ5MRC3XEMGLTIJ7FEPPB2AFHODVA4I2RM6N7Q6SNTLNCBQ


'Escrow address has been funded in transaction with id: ES74ENJ5MRC3XEMGLTIJ7FEPPB2AFHODVA4I2RM6N7Q6SNTLNCBQ'

In [38]:
game_engine_fixed.win_money_refund(client=client, player_id="O")

4:GameEngineService.win_money_refund
2:ApplicationTransactionRepository.call_application
2:get_default_suggested_params
2:PaymentTransactionRepository.payment
2:get_default_suggested_params
The winning money have been refunded to the player O in the transaction with id: CBPTSQEUU3MDIWKSZXPOMLOH4OFTXTOKDPF6ODSYXE377FUPXMMQ


'The winning money have been refunded to the player O in the transaction with id: CBPTSQEUU3MDIWKSZXPOMLOH4OFTXTOKDPF6ODSYXE377FUPXMMQ'